In [1]:
from numba import njit, prange
import numpy as np
import pandas as pd
import tifffile as tiff
import zarr
import napari
import random
import dask.array as da
from dask.cache import Cache
import zarr
import os
import numpy as np
from pathlib import Path
from magicgui import magicgui
import ast
import distinctipy 
import enum

In [2]:
viewer = napari.Viewer()

In [3]:
@magicgui(call_button='Open image')
def open_image(impath = Path(), channel_list=Path()):
    image = tiff.TiffFile(impath, is_ome = False)
    z = zarr.open(image.aszarr(), mode='r')
    n_levels = len(image.series[0].levels) # pyramid

    # If and if not pyramids are available
    if n_levels > 1:
        pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
        multiscale = True
    else:
        pyramid = da.from_zarr(z)
        multiscale = False 
        
    if channel_list == '.':
        viewer.add_image(pyramid, multiscale=multiscale, channel_axis=0, visible=False, contrast_limits=(0,65535))
    else:
        list_df = pd.read_excel(channel_list)
        clist = list(list_df.loc[:, 'Channel_name'])
        viewer.add_image(pyramid, multiscale=multiscale, channel_axis=0, visible=False, contrast_limits=(0,65535), name=clist)

viewer.window.add_dock_widget(open_image)

In [4]:
@njit(parallel=True)
def filter_array(m_array, mapping_array):
    array = m_array.copy()
    rows, cols = array.shape
    for i in prange(rows):
        for j in range(cols):
            if array[i, j] != 0:
                array[i, j] = mapping_array[array[i, j]]
    return array

In [5]:
@magicgui(call_button = 'Show celltype masks')
def show_celltypes(mask_path = Path(), labels = Path()):
    image = tiff.TiffFile(mask_path, is_ome=False)
    z = zarr.open(image.aszarr(), mode='r')
    mask_array = z[...]
    c_table = pd.read_csv(labels)
    cts = c_table['GlobalCellType'].unique()
    ids = c_table['Cellid'].unique()
    colors = distinctipy.get_colors(len(cts))
    for celltype in range(len(cts)):
        color = list(colors[celltype])
        color.append(1)
        print(cts[celltype])
        filtered_cellIDs = c_table.loc[c_table['GlobalCellType'] == cts[celltype], 'Cellid'].values 
        mapping_array = np.zeros(ids.max() + 2, dtype=np.int32)
        mapping_array[filtered_cellIDs] = 1
        array_filtered = filter_array(mask_array, mapping_array)
        array_filtered = np.where(mask_array == 0, 0, mapping_array[mask_array]) 
        viewer.add_labels(array_filtered, color={0:'black', 1:color}, name= cts[celltype] + '_mask', blending = 'additive')  
        
viewer.window.add_dock_widget(show_celltypes)

Out.of.core
Stromal2
CD68.MP
Stromal1
CD11c.MY
T.regs
B.cells
CD15.MY
Other.immune
CD163.MP
CD4.T.cells
CD8.T.cells
Other.MY
Cancer


### Prepare probabilities column in core's cell types file

#### Available cores from cells_predictions and their molecular profile:
| ImageID             | Molecular Profile |
|---------------------|-------------------|
| TMA_42_961_core100  | BRCAmut/met       |
| TMA_42_961_core108  | BRCAmut/met       |
| TMA_42_961_core118  | BRCAmut/met       |
| TMA_42_961_core127  | BRCAmut/met       |
| TMA_42_961_core104  | BRCAmut/met       |
| TMA_42_961_core114  | BRCAmut/met       |
| TMA_42_961_core91   | BRCAmut/met       |
| TMA_42_961_core97   | BRCAmut/met       |
| TMA_42_961_core27   | BRCAmut/met       |
| TMA_42_961_core36   | BRCAmut/met       |
| TMA_42_961_core48   | BRCAmut/met       |
| TMA_42_961_core57   | BRCAmut/met       |
| TMA_42_961_core68   | BRCAmut/met       |
| TMA_43_616_core78   | BRCAmut/met       |
| TMA_43_616_core79   | BRCAmut/met       |
| TMA_43_616_core80   | BRCAmut/met       |
| TMA_43_616_core83   | BRCAmut/met       |
| TMA_44_810_core16   | BRCAmut/met       |
| TMA_44_810_core18   | BRCAmut/met       |
| TMA_44_810_core22   | BRCAmut/met       |
| TMA_44_810_core23   | BRCAmut/met       |
| TMA_44_810_core41   | BRCAmut/met       |
| TMA_44_810_core44   | BRCAmut/met       |
| TMA_44_810_core46   | BRCAmut/met       |
| TMA_44_810_core45   | BRCAmut/met       |
| TMA_44_810_core47   | BRCAmut/met       |
| TMA_44_810_core48   | BRCAmut/met       |
| TMA_44_810_core51   | BRCAmut/met       |
| TMA_42_961_core107  | HRP               |
| TMA_42_961_core117  | HRP               |
| TMA_42_961_core94   | HRP               |
| TMA_42_961_core99   | HRP               |
| TMA_42_961_core119  | HRP               |
| TMA_42_961_core95   | HRP               |
| TMA_43_616_core55   | HRP               |
| TMA_43_616_core59   | HRP               |
| TMA_43_616_core62   | HRP               |
| TMA_43_616_core63   | HRP               |


In [7]:
def prep_core_df(stromal_cells_prob_df,core):
    # Create a new column 'cell' with the extracted values
    stromal_cells_prob_df['core'] = stromal_cells_prob_df['CellId'].str.extract(r'_core(\d+)_')

    stromal_cells_prob_df = stromal_cells_prob_df[stromal_cells_prob_df['core'] == core]
    stromal_cells_prob_df = stromal_cells_prob_df[['CellId','proba_0']]
    stromal_cells_prob_df.head(5)

    # Create a new column 'cell' with the extracted values
    stromal_cells_prob_df['Cellid'] = stromal_cells_prob_df['CellId'].str.extract(r'_c(\d+)')

    # Convert the 'cell' column to numeric (integer) data type
    stromal_cells_prob_df['Cellid'] = pd.to_numeric(stromal_cells_prob_df['Cellid'], errors='coerce')

    return stromal_cells_prob_df

# Define cell predictions file location
file_name = r"P:\h345\afarkkilab\Projects\NKI\ML_results\Stromal_cells\cells_predictions.csv"
stromal_cells_prob_df = pd.read_csv(file_name)

# Read core's cell type file: CHANGE CORE NUMBER
cell_types_df = pd.read_csv(r"P:\h345\afarkkilab\Projects\NKI\TMA_42_961\Tribus_Cell-segment2\core95_cellTypesNapari.csv")

# Add prob column to core's cell type file: CHANGE CORE NUMBER
stromal_cells_prob_df = prep_core_df(stromal_cells_prob_df,'95')
# Merge the two DataFrames based on the 'cell' column
merged_df = cell_types_df.merge(stromal_cells_prob_df[['Cellid','proba_0']], on='Cellid', how='left')

# Save file: CHANGE CORE NUMBER
merged_df.to_csv(r"P:\h345\afarkkilab\Projects\NKI\TMA_42_961\Tribus_Cell-segment2\core95_cellTypesNapari_prob.csv")

### Add widget for visualising probabilities of cells being BRCAmut

In [8]:
class Marker(enum.Enum):
    prob = 'proba_0'

@magicgui(call_button='Get Marker Intensity Mask', layout='vertical')
def get_marker_intensity_mask(marker: Marker, mask=Path(), marker_intensity_table=Path()):
    table_name = str(marker_intensity_table)
    whole_mask_name = Path(mask).stem

    df = pd.read_csv(table_name)
    df_core = df[df.Core_Names == whole_mask_name]
    df_core_cellid = df_core.set_index('Cellid')

    image = tiff.TiffFile(mask, is_ome=False)
    z = zarr.open(image.aszarr(), mode='r')
    whole_mask = z[...]

    # Initialize a fill value that is outside the range of your "prob" values
    fill_value = -1.0

    # Initialize an array to hold the marker intensity mask
    MarkerMask = np.zeros(shape=whole_mask.shape, dtype=np.float64)

    rows = whole_mask.shape[0]
    cols = whole_mask.shape[1]

    # Loop through each pixel in the image
    for i in range(rows):
        for j in range(cols):
            id = whole_mask[i, j]
            # Check if the id is in the index of the DataFrame
            if id in df_core_cellid.index:
                prob_value = df_core_cellid[marker.value][id]
                # Check if the prob_value is a valid floating-point number
                if isinstance(prob_value, np.float64) and not np.isnan(prob_value):
                    # Assign the prob_value to the corresponding location in MarkerMask
                    MarkerMask[i, j] = prob_value
    
    # Apply colormap only to cells with non-NaN values using masking
    viewer.add_image(
        np.expand_dims(MarkerMask, axis=0),
        name=(marker.value + '_intensity'),
        colormap='inferno',
        scale=(1, 1), 
        visible=True,
    )

viewer.window.add_dock_widget(get_marker_intensity_mask, area='right')